## Step 1 - Scraping

In [1]:
# dependencies
import pandas as pd
import datetime as dt
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

def get_soup(url):
    # Initialize Browser
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    # Visit URL
    browser.visit(url)

    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    return browser, soup

### Nasa Mars News 
1. Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. 
2. Assign the text to variables that you can reference later.

```python
Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```

In [2]:
# set url
nmn_url = 'https://redplanetscience.com/'

# get soup
browser, soup = get_soup(nmn_url)

# Retrieve news element
news = soup.find('div', id='news')

# get all list dates
list_dates = news.find_all('div', class_='list_date')

maxdate = max([dt.datetime.strptime(ldate.text.strip(), '%B %d, %Y') for ldate in list_dates])

# get articles
articles = news.find_all('div', class_='list_text')

for article in articles:
    list_date = article.select_one(".list_date").text.strip()

    if dt.datetime.strptime(list_date, '%B %d, %Y') == maxdate:
        newsa = article
        news_title = newsa.select_one(".content_title").text.strip()
        news_p = newsa.select_one(".article_teaser_body").text.strip()

# prepare output
news_dict = {'title':news_title,
                'lead_in':news_p}

browser.quit()

news_dict

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Erin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


{'title': "The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover",
 'lead_in': 'Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.'}

### JPL Mars Space Images - Featured Image

* Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

```python
# Example:
featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'
```

In [3]:
# set url
spcimages_url = 'https://spaceimages-mars.com/'

# get soup
browser, soup = get_soup(spcimages_url)

featured_image = soup.find('img', class_='headerimage fade-in')

featured_image_url = spcimages_url + featured_image['src']

browser.quit()

featured_image_url

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Erin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

* Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [8]:
mf_url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(mf_url)
df = tables[0]

df.columns = ['','Mars','Earth']
df = df.iloc[1:]
df = df.replace(':','',regex=True)
# df.set_index('', inplace=True)


df.head()

,,Mars,Earth
1,Diameter,"6,779 km","12,742 km"
2,Mass,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons,2,1
4,Distance from Sun,"227,943,824 km","149,598,262 km"
5,Length of Year,687 Earth days,365.24 days


### Mars Hemispheres

* Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```

In [5]:
# set url
hms_url = 'https://marshemispheres.com/'

# get soup
browser, soup = get_soup(hms_url)

items = soup.find_all('div', class_="item")

hemisphere_image_urls = []

for item in items:
    hmsphere = {}
    name = item.h3.text
    link = item.a['href']

    # get full image
    try:
        browser.links.find_by_partial_text(name).click()
        print(browser.url)
        html2 = browser.html
        imgsoup = BeautifulSoup(html2, 'html.parser')
        imgsoup
        img = imgsoup.find('img', class_="wide-image")
        
        hmsphere['title'] = name[:-9]
        hmsphere['img_url'] = hms_url + img['src']
          
    except:
        print("Could not get Image Link")   
        
    hemisphere_image_urls.append(hmsphere)    
    browser.back()

browser.quit()

print(hemisphere_image_urls)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Erin\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html
[{'title': 'Cerberus Hemisphere', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


## Hints

* Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

* Use Pymongo for CRUD applications for your database. For this homework, you can simply overwrite the existing document each time the `/scrape` url is visited and new data is obtained.

* Use Bootstrap to structure your HTML template.